Hello\! As an AWS Solutions Architect, I’ve designed this workflow to be streamlined and cost-effective, leveraging the serverless nature of **Amazon Athena** and **SageMaker AI**.

Since you are running this as an admin, you already have the necessary IAM permissions to create buckets and run crawlers. Here is your step-by-step implementation guide.

-----

## Phase 1: Data Lake Setup (S3 & Glue)

### 1\. Create the S3 Bucket

  * Navigate to the **S3 Console**.
  * Click **Create bucket**.
  * **Bucket name:** `olist-analytics-data-[your-account-id]` (must be globally unique).
  * Keep default settings (Block all public access should be **on**).

### 2\. Upload OLIST CSV Files

  * Organize your local OLIST files into folders if possible (e.g., `orders/`, `products/`, `customers/`).
  * In the S3 console, enter your bucket and click **Upload**.
  * Drag and drop the CSV files.
    > **Architect's Tip:** Ensure your folder structure is clean, as the Glue Crawler will use the folder names to define the table names in Athena.

### 3\. Create Athena Database via Glue Crawler

We will use a Crawler to automatically infer the schema from your CSVs.

  * Go to **AWS Glue Console** -\> **Crawlers** -\> **Create crawler**.
  * **Name:** `olist-crawler`.
  * **Data source:** Add your S3 bucket path (e.g., `s3://olist-analytics-data-.../`).
  * **IAM Role:** Create a new role (Glue will auto-generate one for you).
  * **Output Database:** Click **Add database** and name it `olist_db`.
  * **Schedule:** Set to **On demand**.
  * **Run Crawler:** Once created, select the crawler and click **Run**. This will populate the `olist_db` with tables corresponding to your CSVs.

-----

## Phase 2: Analytics Environment (SageMaker AI)

### 4\. Open a SageMaker Serverless Notebook

AWS has transitioned the notebook experience. We will use the **SageMaker AI** (formerly SageMaker Studio) interface.

1.  Navigate to **Amazon SageMaker** in the AWS Console.
2.  Click on **SageMaker AI** (or Studio) in the left sidebar.
3.  Under the **Applications** section, select **JupyterLab**.
4.  Create or Open a **Space**.
5.  Inside the JupyterLab interface, you are running in a managed, serverless environment. Simply open a new `.ipynb` file using the **Python 3 (Data Science)** kernel.

-----

## Phase 3: Notebook Implementation

### 5\. Starter Code: Connection & Configuration

You will need the `PyAthena` library. If it's not installed in your environment, run `!pip install pyathena` in the first cell.

In [1]:
import pandas as pd
from pyathena import connect

# Configuration
REGION = 'us-east-1' # Change to your region
S3_STAGING_DIR = 's3://your-athena-query-results-bucket/' # Athena needs a place to store query metadata
DATABASE = 'olist_db'

# Establish Connection
conn = connect(s3_staging_dir=S3_STAGING_DIR,
               region_name=REGION)

### 6\. Sample SQL Queries (Group By & Joins)

These queries help you understand order volume and customer distribution.

In [2]:
# Query 1: Top 5 Cities by Order Count (Group By)
query_1 = f"""
SELECT
    customer_city,
    count(customer_id) as total_orders
FROM "{DATABASE}"."customers"
GROUP BY customer_city
ORDER BY total_orders DESC
LIMIT 5;
"""

# Query 2: Joining Orders and Payments to find Revenue by Payment Type (Join)
query_2 = f"""
SELECT
    p.payment_type,
    SUM(p.payment_value) as total_revenue
FROM "{DATABASE}"."order_payments" p
JOIN "{DATABASE}"."orders" o ON p.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY p.payment_type
ORDER BY total_revenue DESC;
"""

### 7\. Import Results into Pandas DataFrame

The most efficient way to handle this in SageMaker is to use the `pd.read_sql` function.

In [ ]:
# Function to execute and return DataFrame
def get_athena_data(query):
    return pd.read_sql(query, conn)

# Load Top Cities into DF
df_cities = get_athena_data(query_1)

# Load Revenue Analysis into DF
df_revenue = get_athena_data(query_2)

# Display results
print("Top Cities Analysis:")
print(df_cities.head())

print("\nRevenue by Payment Type:")
print(df_revenue.head())

In [9]:
import sagemaker
print(sagemaker.get_execution_role())

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::254283132378:role/service-role/AmazonSageMaker-ExecutionRole-20251215T191125


Copy get_athena_dataget_athena_data the Role Name: From your error message, the role is AmazonSageMaker-ExecutionRole-20251215T191125.

Go to IAM Console: Navigate to Roles and search for that name.

Add Permissions: * Click Add permissions -> Attach policies.

Search for and attach the managed policy: AWSGlueConsoleFullAccess.

Note: This unblocks glue:GetDatabase, glue:GetTable, and other metadata actions Athena requires.

2. Grant the Permission in IAM
Go to the IAM Console in a new tab.

Click Roles on the left and search for the role name you just copied.

Click on the Role name.

Click Add permissions -> Attach policies.

Since you are an admin and likely want to move fast, search for AmazonS3FullAccess and check the box.

Architect's Note: In a production environment, we'd use "Least Privilege" (only the specific bucket), but for an OLIST sandbox with an admin user, AmazonS3FullAccess is the fastest way to unblock your workflow.

Also, search for and attach AmazonAthenaFullAccess just to be safe.

Use **SQL Magic commands**. This is often preferred by Data Engineers because it allows you to write raw SQL in a cell without wrapping it in Python strings, providing better syntax highlighting and readability.

To do this, we use the `ipython-sql` extension and the `PyAthena` driver.

### 1\. Install and Load the Extension

Run this in your first notebook cell to set up the environment:

In [11]:
# Install the necessary libraries
!pip install ipython-sql pyathena --quiet

# Load the SQL magic extension
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### 2\. Connect to Athena

Instead of a standard Python connection object, you will pass a connection string to the `%sql` magic command. Replace the placeholders with your specific S3 bucket (created in Step 1) and your region.

In [12]:
import urllib.parse

# Configuration
region = "us-east-1"
athena_db = "olist_db"
# S3 path for Athena to store query results (must end with a /)
s3_output = "s3://olist-analytics-data-results/output/"

# Encode the S3 path for the connection string
connection_string = f"awsathena+rest://@athena.{region}.amazonaws.com/{athena_db}?s3_staging_dir={urllib.parse.quote_plus(s3_output)}"

# Connect the magic command to Athena
%sql $connection_string

In [16]:
# Fix the PrettyTable version conflict
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [17]:
%%sql
SHOW TABLES IN olist_db;

 * awsathena+rest://@athena.us-east-1.amazonaws.com/olist_db?s3_staging_dir=s3%3A%2F%2Folist-analytics-data-results%2Foutput%2F
Done.


tab_name
customers
geolocation
order_items
order_payments
order_reviews
orders
product_category_name_translation
products
sellers


In [26]:
#%%sql
#DESCRIBE order_items;

In [23]:
%%sql
select * from order_items
limit 10

Done.


order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
"""00010242fe8c5a6d1ba2dd792cb16214""",1,"""4244733e06e7ecb4970a6e2683c13e61""","""48436dade18ac8b2bce089ec2a041202""",2017-09-19 09:45:35,58.9,13.29
"""00018f77f2f0320c557190d7a144bdd3""",1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
"""000229ec398224ef6ca0657da4fc703e""",1,c777355d18b72b67abbeef9df44fd0fd,"""5b51032eddd242adc84c38acab88f23d""",2018-01-18 14:48:30,199.0,17.87
"""00024acbcdf0a6daa1e931b038114c75""",1,"""7634da152a4610f1595efa32f14722fc""","""9d7a1d34a5052409006425275ba1c2b4""",2018-08-15 10:10:18,12.99,12.79
"""00042b26cf59d7ce69dfabb4e55b4fd9""",1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9,18.14
"""00048cc3ae777c65dbb7d2a0634bc1ea""",1,ef92defde845ab8450f9d70c526ef70f,"""6426d21aca402a131fc0a5d0960a3c90""",2017-05-23 03:55:27,21.9,12.69
"""00054e8431b9d7675808bcb819fb4a32""",1,"""8d4f2bb7e93e6710a28f34fa83ee7d28""","""7040e82f899a04d1b434b795a43b4617""",2017-12-14 12:10:31,19.9,11.85
"""000576fe39319847cbb9d288c5617fa6""",1,"""557d850972a7d6f792fd18ae1400d9b6""","""5996cddab893a4652a15592fb58ab8db""",2018-07-10 12:30:45,810.0,70.75
"""0005a1a1728c9d785b8e2b08b904576c""",1,"""310ae3c140ff94b03219ad0adc3c778f""",a416b6a846a11724393025641d4edd5e,2018-03-26 18:31:29,145.95,11.65
"""0005f50442cb953dcd1d21e1fb923495""",1,"""4535b0e1091c278dfd193e5a1d63b39f""",ba143b05f0110f0dc71ad71b4466ce92,2018-07-06 14:10:56,53.99,11.4


In [ ]:
%%sql
/* Find the top 5 product categories by number of items sold */
SELECT
    product_category_name,
    COUNT(*) as items_sold
FROM order_items
GROUP BY product_category_name
ORDER BY items_sold DESC
LIMIT 5

-----

### 3\. Run SQL Queries Directly in Cells

Now you can use the `%%sql` prefix at the top of any cell to write pure SQL.

**Example: Group By Analysis**

```sql
%%sql
/* Find the top 5 product categories by number of items sold */
SELECT
    product_category_name,
    COUNT(*) as items_sold
FROM olist_order_items_dataset
GROUP BY product_category_name
ORDER BY items_sold DESC
LIMIT 5;
```

**Example: Multi-Table Join**

```sql
%%sql
/* Join orders and customers to see the delivery performance by state */
SELECT
    c.customer_state,
    AVG(date_diff('day', CAST(o.order_purchase_timestamp AS TIMESTAMP), CAST(o.order_delivered_customer_date AS TIMESTAMP))) AS avg_delivery_days
FROM olist_orders_dataset o
JOIN olist_customers_dataset c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
ORDER BY avg_delivery_days ASC
LIMIT 10;
```

-----

Since we are looking at the OLIST dataset—which is a treasure trove of Brazilian e-commerce history—we can dig into more complex "business intelligence" style queries.

Here are three advanced SQL queries using **Window Functions**, **Date Arithmetic**, and **Complex Joins** to run directly in your SageMaker SQL cells.

---

### 1. Customer Retention: Repeat Purchase Behavior

This query identifies "Loyalists" by calculating how many customers have placed more than one order. It requires joining the `orders` table with the `customers` table since `customer_id` in the orders table is unique per order, but `customer_unique_id` is the actual permanent ID for the person.

```sql
%%sql
SELECT
    CASE
        WHEN order_count = 1 THEN 'One-time Buyer'
        WHEN order_count = 2 THEN 'Repeat Buyer (2)'
        ELSE 'Loyal Customer (3+)'
    END AS customer_segment,
    COUNT(*) as customer_count
FROM (
    SELECT
        c.customer_unique_id,
        COUNT(o.order_id) as order_count
    FROM olist_orders_dataset o
    JOIN olist_customers_dataset c ON o.customer_id = c.customer_id
    GROUP BY c.customer_unique_id
)
GROUP BY 1
ORDER BY customer_count DESC;

```

---

### 2. Seasonality: Monthly Revenue Growth (%)

This is a more advanced query using the **Window Function** `LAG()` to compare the current month's revenue to the previous month. This is a classic "Solution Architect" pattern for building executive dashboards.

```sql
%%sql
WITH MonthlyRevenue AS (
    SELECT
        date_format(CAST(o.order_purchase_timestamp AS TIMESTAMP), '%Y-%m') AS order_month,
        SUM(p.payment_value) AS revenue
    FROM olist_orders_dataset o
    JOIN olist_order_payments_dataset p ON o.order_id = p.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY 1
)
SELECT
    order_month,
    revenue,
    LAG(revenue) OVER (ORDER BY order_month) AS prev_month_revenue,
    (revenue - LAG(revenue) OVER (ORDER BY order_month)) / LAG(revenue) OVER (ORDER BY order_month) * 100 AS pct_growth
FROM MonthlyRevenue
ORDER BY order_month;

```

---

### 3. Logistic Bottlenecks: Estimated vs. Actual Delivery

One of the biggest challenges in Brazilian e-commerce is geography. This query finds the states where the logistics gap (Difference between when the customer was *told* it would arrive vs. when it *actually* arrived) is the highest.

```sql
%%sql
SELECT
    c.customer_state,
    COUNT(o.order_id) as total_orders,
    AVG(date_diff('day', CAST(o.order_delivered_customer_date AS TIMESTAMP), CAST(o.order_estimated_delivery_date AS TIMESTAMP))) AS days_ahead_of_schedule,
    SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS percent_late
FROM olist_orders_dataset o
JOIN olist_customers_dataset c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
GROUP BY c.customer_state
HAVING COUNT(o.order_id) > 100
ORDER BY percent_late DESC;

```

---

### Tips for Athena Performance in SageMaker

* **Case Sensitivity:** Athena tables and columns are usually lowercase. If your CSV headers had caps, the Glue Crawler might have lowercased them automatically.
* **Date Casting:** OLIST timestamps are often stored as strings in CSVs. Notice I used `CAST(column AS TIMESTAMP)` in the queries above; this is essential for `date_diff` or `date_format` to work.

**Next Step:** Would you like to see how to use **SageMaker Clarify** on these results to check for bias in your delivery times across different Brazilian states?
-----


### 4\. Convert SQL Cell Results to Pandas

One of the best features of SQL magic is that you can "capture" the output of a SQL cell directly into a Pandas DataFrame using the assignment syntax.

In [ ]:
# Use the << operator to assign the result of the SQL query to a variable named 'df_performance'
%%sql df_performance <<
SELECT
    order_status,
    COUNT(*) as count
FROM olist_orders_dataset
GROUP BY order_status

In [ ]:
# Now 'df_performance' is a standard Pandas DataFrame
print(type(df_performance))
df_performance.head()

-----

**Next Step:** Would you like to see how to create a **SageMaker Feature Store** from these SQL results to prepare the data for an ML model?

As an AWS Solutions Architect, I recommend using **Seaborn** for visualizations in SageMaker because it handles Pandas DataFrames natively and produces "production-ready" aesthetics with very little code.

Below is the code to pull the data from Athena into Pandas and generate three key business insights: **Revenue Trends**, **Payment Preferences**, and **Category Performance**.

### 1\. Setup and Library Imports

Run this in a new cell to prepare your plotting environment.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set the visual style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

-----

### 2\. Visualization 1: Monthly Revenue Trend (Growth Analysis)

We will pull the monthly revenue to see how the business is scaling.

In [ ]:
# Query data into a DataFrame
df_revenue_trend = get_athena_data(f"""
    SELECT
        date_format(CAST(o.order_purchase_timestamp AS TIMESTAMP), '%Y-%m-01') AS month,
        SUM(p.payment_value) AS total_revenue
    FROM "{DATABASE}"."olist_orders_dataset" o
    JOIN "{DATABASE}"."olist_order_payments_dataset" p ON o.order_id = p.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY 1
    ORDER BY 1
""")

# Convert month string to datetime for better plotting
df_revenue_trend['month'] = pd.to_datetime(df_revenue_trend['month'])

# Plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_revenue_trend, x='month', y='total_revenue', marker='o', color='#2ecc71')
plt.title('OLIST Monthly Revenue Trend (BRL)', fontsize=15)
plt.xlabel('Order Month')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.show()

-----

### 3\. Visualization 2: Payment Method Distribution

This helps identify the most popular financial rails in the Brazilian market.

In [ ]:
# Query data
df_payments = get_athena_data(f"""
    SELECT payment_type, COUNT(*) as count
    FROM "{DATABASE}"."olist_order_payments_dataset"
    GROUP BY payment_type
    ORDER BY count DESC
""")

# Plotting a Pie Chart (better for distribution)
plt.figure(figsize=(8, 8))
colors = sns.color_palette('pastel')[0:5]
plt.pie(df_payments['count'], labels=df_payments['payment_type'], autopct='%1.1f%%', colors=colors, startangle=140)
plt.title('Customer Payment Method Distribution', fontsize=15)
plt.show()

-----

### 4\. Visualization 3: Top 10 Product Categories by Revenue

This join-heavy query identifies which categories drive the most value.

In [ ]:
# Query data
df_top_categories = get_athena_data(f"""
    SELECT
        p.product_category_name,
        SUM(i.price) as total_sales
    FROM "{DATABASE}"."olist_order_items_dataset" i
    JOIN "{DATABASE}"."olist_products_dataset" p ON i.product_id = p.product_id
    WHERE p.product_category_name IS NOT NULL
    GROUP BY p.product_category_name
    ORDER BY total_sales DESC
    LIMIT 10
""")

# Plotting a Horizontal Bar Chart
sns.barplot(data=df_top_categories, x='total_sales', y='product_category_name', palette='viridis')
plt.title('Top 10 Product Categories by Revenue', fontsize=15)
plt.xlabel('Total Sales (BRL)')
plt.ylabel('Category Name')
plt.tight_layout()
plt.show()

-----

### 5\. Architectural Tip: Handling "Big Data" in SageMaker

If your OLIST dataset grows to millions of rows:

1.  **Sampling:** Use `TABLESAMPLE BERNOULLI (10)` in your SQL query to pull only a 10% representative sample into Pandas to save memory.
2.  **Parquet Conversion:** In Step 3 (Crawler), if you convert your CSVs to **Parquet** format in S3, your Athena queries will run up to **10x faster** and cost **90% less** because Athena only reads the columns needed for the visualization.

**Next Step:** Would you like to build a **Linear Regression model** in this notebook to predict future delivery times based on the geographic data we just visualized?